# UZOP projekt 2022./23.
Ova Jupyter bilježnica rezultat je projektnog rada u sklopu kolegija Uvod u znanost podacima na FER-u, ak. god. 2022./23. U sklopu projektnog zadatka zadan je članak s određenom temom iz područja znanosti o podacima, skupa sa svojim podacima koje je potrebno pripremiti, pomoću njih replicirati rezultate rada u članku te naposlijetku predložiti eventualna poboljšanja. U poglavljima koja slijede svi su ovi koraci prikazani uz prikladna objašnjenja.

Autor članka "Predicting NBA shots" na kojem se temelji ovaj projekt je Brett Meehan sa sveučilišta Stanford u SAD-u. Članak je javno dostupan na poveznici: http://cs229.stanford.edu/proj2017/final-reports/5132133.pdf. Članak istražuje primjenu različitih algoritama strojnog učenja na problem predviđanja uspješnosti bacanja igrača u NBA ligi.

## 1. Priprema i vizualizacija podataka
Prije bilo kakvog rada s podacima, potrebno je upoznati se s njima kako bismo ih mogli ispravno prikazati, koristiti i na temelju njih nešto zaključiti. Korištenje tzv. sirovih podataka u statističkoj analizi bez dubljeg promišljanja o njima može prouzročiti razne probleme, samo neki od kojih su neispravno postavljanje cilja analize, rušenje algoritama strojnog učenja te na koncu i donošenje neispravnih zaključaka.

Priprema podataka podrazumijeva njihovu pripremu za daljnju analizu. Tek nakon provedene pripreme možemo podatke organizirati i eventualno upotpuniti ukoliko ima nedostajućih vrijednosti. Naposlijetku je podatke poželjno vizualizirati.

### 1.1 Učitavanje osnovnih biblioteka


In [1]:
import numpy as np
import pandas as pd

### 1.2 Učitavanje podataka
Podaci su nam dostupni u obliku CSV datoteke `shot_logs.csv`.

In [2]:
dataset = pd.read_csv("shot_logs.csv")

### 1.3 Početni pregled podataka

Pomoću atributa `shape` možemo doznati dimenzije skupa podataka - u našem slučaju, to su dvije dimenzija: broj redaka (primjera) i broj stupaca (značajki).

In [3]:
dataset.shape

(128069, 21)

Funkcija `head(n=5)` iz paketa `Pandas` omogućuje nam prikaz prvih `n` redaka iz skupa podataka.

In [21]:
dataset.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.8,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.4,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,NaN,3,...,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.3,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.9,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


Kako naš skup podataka ima dvije dimenzije, njegovo svojstvo `columns` ima atribut `values` preko kojeg doznajemo nazive svih značajki u našem skupu podataka. Ovo nam je važno jer ćemo neke značajke morati izbaciti, a neke će nam pak biti važnije od ostalih u predviđanjima nad skupom podataka.

In [6]:
dataset.columns.values

array(['GAME_ID', 'MATCHUP', 'LOCATION', 'W', 'FINAL_MARGIN',
       'SHOT_NUMBER', 'PERIOD', 'GAME_CLOCK', 'SHOT_CLOCK', 'DRIBBLES',
       'TOUCH_TIME', 'SHOT_DIST', 'PTS_TYPE', 'SHOT_RESULT',
       'CLOSEST_DEFENDER', 'CLOSEST_DEFENDER_PLAYER_ID', 'CLOSE_DEF_DIST',
       'FGM', 'PTS', 'player_name', 'player_id'], dtype=object)

U nastavku su dana pojašnjenja naziva pojedinih značajki:
- `GAME_ID` - identifikacijska oznaka utakmice
- `MATCHUP` - "naziv" utakmice, koji uključuje datum i imena suprotstavljenih timova
- `LOCATION` - lokacija na kojoj je odigrana utakmica, može biti `H` (_home_) ili `A` (_away_)
- `W` - ishod utakmice, može biti `W` (_win_) ili `L` (_loss_)
- `FINAL_MARGIN` - razlika u ostvarenim poenima između pobjedničkog i gubitničkog tima na kraju utakmice
- `SHOT_NUMBER` - redni broj pucanja za dotičnog igrača
- `PERIOD` - dio utakmice, može biti prirodan broj iz skupa {1, 2, 3, 4} i označava redni broj četvrtine
- `GAME_CLOCK` - vrijeme u trenutku pucanja na satu koji označava preostalo vrijeme do kraja trenutne četvrtine
- `SHOT_CLOCK` - vrijeme u trenutku pucanja na satu koji označava preostalo vrijeme za napad
- `DRIBBLES` - broj driblanja igrača prije pucanja na koš
- `TOUCH_TIME` - mjera vremena koliko je lopta provela u igračevom posjedu
- `SHOT_DIST` - udaljenost mjesta pucanja od koša
- `PTS_TYPE` - vrsta pucanja koja zapravo označava broj poena koje igrač osvaja za svoj tim u slučaju pogotka, može biti prirodan broj iz skupa {2, 3}
- `SHOT_RESULT` - ishod pucanja, može biti `made` (pogodak) ili `missed` (promašaj)
- `CLOSEST_DEFENDER` - ime protivničkog obrambenog igrača koji se u trenutku pucanja nalazio najbliže igraču koji puca
- `CLOSEST_DEFENDER_PLAYERID` - id protivničkog obrambenog igrača koji se u trenutku pucanja nalazio najbliže igraču koji puca
- `CLOSE_DEF_DIST` - udaljenost najbližeg protivničkog obrambenog igrača od igrača koji puca
- `FGM` - označava je li postignut pogodak koji nije iz slobodnog bacanja, može biti `1` (pogodak postignut) ili `0` (pogodak nije postignut)
- `PTS` - ostvareni broj poena za pogodak, može biti cijeli broj iz skupa {0, 1, 2, 3}
- `player_name` - ime igrača koji puca
- `player_id` - identifikacijska oznaka igrača koji puca

U nastavku su pomoću funkcije `describe()` dane neke osnovne deskriptivne statistike nad skupom podataka.

In [12]:
dataset.describe()

,GAME_ID,FINAL_MARGIN,SHOT_NUMBER,PERIOD,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DIST,PTS_TYPE,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_id
count,1.280690e+05,128069.000000,128069.000000,128069.000000,122502.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000,128069.000000
mean,2.140045e+07,0.208723,6.506899,2.469427,12.453344,2.023355,2.765901,13.571504,2.264670,159038.487284,4.123015,0.452139,0.997314,157238.251247
std,2.578773e+02,13.233267,4.713260,1.139919,5.763265,3.477760,3.043682,8.888964,0.441159,78791.172947,2.756446,0.497706,1.130978,79362.389336
min,2.140000e+07,-53.000000,1.000000,1.000000,0.000000,0.000000,-163.600000,0.000000,2.000000,708.000000,0.000000,0.000000,0.000000,708.000000
25%,2.140023e+07,-8.000000,3.000000,1.000000,8.200000,0.000000,0.900000,4.700000,2.000000,101249.000000,2.300000,0.000000,0.000000,101162.000000
50%,2.140045e+07,1.000000,5.000000,2.000000,12.300000,1.000000,1.600000,13.700000,2.000000,201949.000000,3.700000,0.000000,0.000000,201939.000000
75%,2.140067e+07,9.000000,9.000000,3.000000,16.675000,2.000000,3.700000,22.500000,3.000000,203079.000000,5.300000,1.000000,2.000000,202704.000000
max,2.140091e+07,53.000000,38.000000,7.000000,24.000000,32.000000,24.900000,47.200000,3.000000,530027.000000,53.200000,1.000000,3.000000,204060.000000


<div style="color:red">Potrebno je iz skupa podataka izbaciti __monotone atribute__, odnosno one čija vrijednost jednoliko raste, jer nam oni ne donose nikakvu vrijednost u predviđanju.</div>

In [13]:
dataset.nunique()

GAME_ID                        904
MATCHUP                       1808
LOCATION                         2
W                                2
FINAL_MARGIN                    88
SHOT_NUMBER                     38
PERIOD                           7
GAME_CLOCK                     719
SHOT_CLOCK                     241
DRIBBLES                        33
TOUCH_TIME                     313
SHOT_DIST                      448
PTS_TYPE                         2
SHOT_RESULT                      2
CLOSEST_DEFENDER               473
CLOSEST_DEFENDER_PLAYER_ID     474
CLOSE_DEF_DIST                 299
FGM                              2
PTS                              3
player_name                    281
player_id                      281
dtype: int64

### 1.4 Preoblikovanje podataka
U skupu podataka ima nedostajućih vrijednosti, što otežava kasniju obradu i vizualizaciju. Nekoliko je načina razrješavanja nedostajućih vrijednosti u skupu podataka: izbacivanje zapisa iz skupa podataka, izbacivanje značajke iz skupa podataka, zamjena srednjom vrijednošću preostalih vrijednosti, pronalazak točnih vrijednosti na drugim mjestima itd.

Možemo pronaći nedostajuće vrijednosti u našem skupu podataka pomoću funkcije `isna()` koja traži vrijednosti `NaN` u skupu podataka. Broj pojavljivanja takvih vrijednosti pobrojimo po značajkama pomoću funkcije `sum()`.

In [14]:
dataset.isna().sum()

GAME_ID                          0
MATCHUP                          0
LOCATION                         0
W                                0
FINAL_MARGIN                     0
SHOT_NUMBER                      0
PERIOD                           0
GAME_CLOCK                       0
SHOT_CLOCK                    5567
DRIBBLES                         0
TOUCH_TIME                       0
SHOT_DIST                        0
PTS_TYPE                         0
SHOT_RESULT                      0
CLOSEST_DEFENDER                 0
CLOSEST_DEFENDER_PLAYER_ID       0
CLOSE_DEF_DIST                   0
FGM                              0
PTS                              0
player_name                      0
player_id                        0
dtype: int64

Vidimo da u našem skupu podataka značajka `SHOT_CLOCK` ima nedostajućih vrijednosti.

In [26]:
dataset_shotClockNaN = dataset[dataset["SHOT_CLOCK"].isna()]
dataset_shotClockNaN.loc[:, ["GAME_CLOCK", "SHOT_CLOCK"]]

,GAME_CLOCK,SHOT_CLOCK
2,0:00,NaN
24,0:04,NaN
54,0:01,NaN
76,0:01,NaN
129,0:02,NaN
...,...,...
128008,0:02,NaN
128017,0:03,NaN
128018,0:02,NaN
128051,0:16,NaN


Iz navedenog prikaza možemo zaključiti da se _shot clock_ više ne resetira nakon što _game clock_ vrijednost padne ispod 24 sekunde. Nakon što doše do nule, shot clock se ugasi jer igrači od tog trenutka više nemaju 24 sekunde na raspolaganju za napad, već onoliko koliko je preostalo do kraja četvrtine.

Vrijednosti `NaN` u stupcu značajke `SHOT_CLOCK` možemo zamijeniti srednjom vrijednošću preostalih vrijednosti koje nisu `NaN`. Nema smisla izbacivati te zapise iz skupa podataka jer ih ima puno, a značajku pogotovo nema smisla izbacivati jer igrači imaju tendenciju pucati kada je vrijednost na _shot clocku_ mala.

In [30]:
dataset_tmp = dataset.copy()

dataset_tmp.loc[dataset_tmp.SHOT_CLOCK.isna(), 'SHOT_CLOCK'] = dataset_tmp.loc[:, 'SHOT_CLOCK'].mean()
dataset_tmp.head()

,GAME_ID,MATCHUP,LOCATION,W,FINAL_MARGIN,SHOT_NUMBER,PERIOD,GAME_CLOCK,SHOT_CLOCK,DRIBBLES,...,SHOT_DIST,PTS_TYPE,SHOT_RESULT,CLOSEST_DEFENDER,CLOSEST_DEFENDER_PLAYER_ID,CLOSE_DEF_DIST,FGM,PTS,player_name,player_id
0,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,1,1,1:09,10.800000,2,...,7.7,2,made,"Anderson, Alan",101187,1.3,1,2,brian roberts,203148
1,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,2,1,0:14,3.400000,0,...,28.2,3,missed,"Bogdanovic, Bojan",202711,6.1,0,0,brian roberts,203148
2,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,3,1,0:00,12.453344,3,...,10.1,2,missed,"Bogdanovic, Bojan",202711,0.9,0,0,brian roberts,203148
3,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,4,2,11:47,10.300000,2,...,17.2,2,missed,"Brown, Markel",203900,3.4,0,0,brian roberts,203148
4,21400899,"MAR 04, 2015 - CHA @ BKN",A,W,24,5,2,10:34,10.900000,2,...,3.7,2,missed,"Young, Thaddeus",201152,1.1,0,0,brian roberts,203148


Vidimo da u stupcu `SHOT_CLOCK` više nema nedostajućih vrijednosti, već su one sada zamijenjene srednjom vrijednošću nenedostajućih vrijednosti koja iznosi 12.453344 s.